<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deep-daze stomp.py --upgrade

     |████████████████████████████████| 1.4MB 10.1MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 3.3MB 36.1MB/s 
     |████████████████████████████████| 92kB 9.8MB/s 
  Created wheel for ftfy: filename=ftfy-6.0-cp37-none-any.whl size=41622 sha256=cab1592c47332e9949001ad129743ff50148de7fc7ba15f3fed5e3f9bdbb23c4
  Stored in directory: /root/.cache/pip/wheels/22/8b/08/7d1c17849e10371206a262304973b5a9f45e8b9d0a2179f465
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=b18cf889b1d62e3c6320afac99768b771f7fa0d816e94d4a871394a96660ba13
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built ftfy fire
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imagei

In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep

flatten = itertools.chain.from_iterable

WATCH_MEDIA_FILES = ["./*.jpg","./*.mp4"]

conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)


!rm *.jpg

p = Popen('imagine --width=128 --iterations=50 --epochs=5 --save-every=5 --save-progress=True --open_folder=False --overwrite=True "A slick high-contrast"', stdout = PIPE, 
        stderr = STDOUT, shell = True)

FOUND_MEDIA_FILES=[]
while True:
  line = p.stdout.readline()#
  if not line: break

  stdout.write(line)
  conn.send(body=line.decode("utf-8"), destination='/topic/colabOut')
  NEW_FOUND_MEDIA_FILES=list(flatten([glob(search_path) for search_path in WATCH_MEDIA_FILES]))
  NEW_FOUND_MEDIA_FILES.sort(key=os.path.getmtime)
  ADDED_MEDIA_FILES = list(set(NEW_FOUND_MEDIA_FILES)-set(FOUND_MEDIA_FILES))
  FOUND_MEDIA_FILES = NEW_FOUND_MEDIA_FILES

  if len(ADDED_MEDIA_FILES) > 0:
    for added in ADDED_MEDIA_FILES:
      sleep(0.1)
      if len(added) > 0:
        print("reading", added)
        with open(added,'rb') as added_media_file:
          added_media = added_media_file.read()
          base64_encoded_added = f"data:image/jpg;base64,{b64encode(added_media).decode('ascii')}"
          conn.send(body=base64_encoded_added, destination='/topic/colabMediaOut')
conn.disconnect()

epochs:   0%|          | 0/5 [00:00<?, ?it/s]
                                             
reading ./A_slick_high-contrast.jpg
loss: -23.41:   8%|▊         | 4/50 [00:07<01:12,  1.57s/it]
                                             
reading ./A_slick_high-contrast.000001.jpg
loss: -25.27:  18%|█▊        | 9/50 [00:15<01:04,  1.57s/it]
                                             
reading ./A_slick_high-contrast.000002.jpg
loss: -25.53:  28%|██▊       | 14/50 [00:23<00:56,  1.57s/it]
                                             
reading ./A_slick_high-contrast.000003.jpg
loss: -25.40:  38%|███▊      | 19/50 [00:31<00:48,  1.56s/it]
                                             
reading ./A_slick_high-contrast.000004.jpg
loss: -26.11:  48%|████▊     | 24/50 [00:39<00:40,  1.56s/it]
                                             
reading ./A_slick_high-contrast.000005.jpg
loss: -28.11:  58%|█████▊    | 29/50 [00:46<00:32,  1.56s/it]
                                             
reading ./A

In [ ]:
from glob import glob
import requests
import requests
from tqdm import tqdm
!mkdir -p results
for imgfile in tqdm(glob("./A_slick*.jpg")):
  print("Super resolution of", imgfile)
  
  r = requests.post(
    "https://api.deepai.org/api/waifu2x",
    files={
        'image': open(imgfile, 'rb'),
    },
    headers={'api-key': "blalasd"}
  )
  srurl = r.json()["output_url"]
  with open(f"/content/results/{imgfile}.upsampled.jpg", 'wb') as f:
    f.write(requests.get(srurl).content)
  #!echo curl -F f"\'image=@{imgfile}\' -H api-key: https://api.deepai.org/api/torch-srgan" 

In [ ]:
!mkdir -p images
!ffmpeg -r 20 -y -i results/A_slick%*.jpg -filter:v scale=512:-1 images/env_school_flashy.mp4

In [ ]:
!pip install stomp-py